We have a CSV file in the following format. For normal vertex, we tag it as GEO (for geodesic string) and for an arc segment, we are tagging it with a combination of two(2) successive ABE (arc-by-edge)

```
25.270806364429227 51.60333739375988 GEO
25.271009225305253 51.60324801592732 GEO
25.271113964392683 51.603201869139234 GEO
25.2707782569834 51.60228029931405 GEO
25.270673510842762 51.602326451082504 GEO
25.27046820899583 51.60241690493056 ABE
25.270466156352832 51.60261064947953 ABE
25.27033759984866 51.602747027489016 GEO
25.27042237891931 51.60297183557495 GEO
25.270507464783933 51.60319640070589 ABE
25.27069468321789 51.60320703990452 ABE
25.270788613761002 51.60330422209724 GEO
```
Want to convert this to a GML file as follows
```
<gml:patches>
<gml:PolygonPatch>
<gml:exterior>
<gml:Ring>
<gml:curveMember>
<gml:Curve gml:id="gmlID389873">
<gml:segments>
<gml:GeodesicString>
<gml:pos>51.6033373938 25.2708063644</gml:pos>
<gml:pos>51.6032480159 25.2710092253</gml:pos>
<gml:pos>51.6032018691 25.2711139644</gml:pos>
<gml:pos>51.6022802993 25.2707782570</gml:pos>
<gml:pos>51.6023264511 25.2706735108</gml:pos>
<gml:pos>51.6024169049 25.2704682090</gml:pos>
</gml:GeodesicString>
<gml:ArcString>
<gml:pos>51.6024169049 25.2704682090</gml:pos>
<gml:pos>51.6026106495 25.2704661564</gml:pos>
<gml:pos>51.6027470275 25.2703375998</gml:pos>
</gml:ArcString>
<gml:GeodesicString>
<gml:pos>51.6027470275 25.2703375998</gml:pos>
<gml:pos>51.6029718356 25.2704223789</gml:pos>
<gml:pos>51.6031964007 25.2705074648</gml:pos>
</gml:GeodesicString>
<gml:ArcString>
<gml:pos>51.6031964007 25.2705074648</gml:pos>
<gml:pos>51.6032070399 25.2706946832</gml:pos>
<gml:pos>51.6033042221 25.2707886138</gml:pos>
</gml:ArcString>
<gml:GeodesicString>
<gml:pos>51.6033042221 25.2707886138</gml:pos>
<gml:pos>51.6033373938 25.2708063644</gml:pos>
</gml:GeodesicString>
</gml:segments>
</gml:Curve>
</gml:curveMember>
</gml:Ring>
</gml:exterior>
</gml:PolygonPatch>
</gml:patches>
```

In [91]:
import csv
import xml.etree.cElementTree as ET
import xml.dom.minidom
import itertools

In [92]:
GML = 'http://www.opengis.net/gml'
ET.register_namespace('gml', GML) 

root = ET.Element('gml')
patches = ET.SubElement(root, ET.QName(GML, 'patches'))
polypatches = ET.SubElement(patches, ET.QName(GML, 'PolygonPatch'))
exterior = ET.SubElement(polypatches, ET.QName(GML, 'exterior'))
ring = ET.SubElement(exterior, ET.QName(GML, 'Ring'))
curvemember = ET.SubElement(ring, ET.QName(GML, 'curveMember'))
curve = ET.SubElement(curvemember, ET.QName(GML, 'curve'), {'gml:id':'gmlID389873'})

In [93]:
with open('data.csv', 'r') as f:
    lines = f.readlines()

processed = [line.strip().split() for line in lines]

In [94]:
# Group all lines where the vertex type is the same
iterator = itertools.groupby(processed, lambda item: item[2])

# iterating over the result
# element and its group
for element, group in iterator:
    if element == 'GEO':
        geodesicstring = ET.SubElement(curve, ET.QName(GML, 'GeodesicString'))
        for vertex in list(group):
            value = '{},{}'.format(vertex[1], vertex[0])
            ET.SubElement(geodesicstring, ET.QName(GML, 'pos')).text = value
    if element == 'ABE':
        arcstring = ET.SubElement(curve, ET.QName(GML, 'ArcString'))
        ET.SubElement(arcstring, ET.QName(GML, 'pos')).text = last_geo_value
        for vertex in list(group):
            value = '{},{}'.format(vertex[1], vertex[0])
            ET.SubElement(arcstring, ET.QName(GML, 'pos')).text = value


In [95]:
dom = xml.dom.minidom.parseString(ET.tostring(root))
xml_string = dom.toprettyxml(indent='  ')
print(xml_string)

<?xml version="1.0" ?>
<gml xmlns:gml="http://www.opengis.net/gml">
  <gml:patches>
    <gml:PolygonPatch>
      <gml:exterior>
        <gml:Ring>
          <gml:curveMember>
            <gml:curve gml:id="gmlID389873">
              <gml:GeodesicString>
                <gml:pos>51.60333739375988,25.270806364429227</gml:pos>
                <gml:pos>51.60324801592732,25.271009225305253</gml:pos>
                <gml:pos>51.603201869139234,25.271113964392683</gml:pos>
                <gml:pos>51.60228029931405,25.2707782569834</gml:pos>
                <gml:pos>51.602326451082504,25.270673510842762</gml:pos>
              </gml:GeodesicString>
              <gml:ArcString>
                <gml:pos/>
                <gml:pos>51.60241690493056,25.27046820899583</gml:pos>
                <gml:pos>51.60261064947953,25.270466156352832</gml:pos>
              </gml:ArcString>
              <gml:GeodesicString>
                <gml:pos>51.602747027489016,25.27033759984866</gml:pos>
           

In [96]:
with open('output.gml', 'w') as f:
    f.write(xml_string)
